In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ENROLLEES = pd.read_csv('Comma Delimited Data/enrollees.csv').drop(['VERSION'],axis=1)
ENROLLEES = ENROLLEES[['ID','P02HISP','P02SEX', 'P02RACE']]

#### Target Data (outcomes99.csv)

In [3]:
dt = pd.read_csv('Comma Delimited Data/outcomes99.csv').drop(['version'],axis=1)

#### Grouping features names by meaning...

In [4]:
tRPCF = [i for i in dt.columns if i[-4:] =='RPCF']   # Knee and hip replacement status during follow-up
tBLRP = [i for i in dt.columns if i[-4:] =='BLRP']   # Baseline knee or hip replacements
tRPSN = [i for i in dt.columns if i[-4:] =='RPSN']   # Knee or hip replacement seen on follow-up OAI x-ray.
tDAYS = [i for i in dt.columns if i[-4:] =='DAYS']   # Closest OAI contact prior to and after replacement
tVSPR = [i for i in dt.columns if i[-4:] =='VSPR']   # Closest OAI contact prior to replacement
tXRPR = [i for i in dt.columns if i[-4:] =='XRPR']   # Closest OAI visit with x-ray prior to replacement
tXRAF = [i for i in dt.columns if i[-4:] =='XRAF']   # Closest OAI visit with x-ray after the replacement

tVSAF = [i for i in dt.columns if i[-4:] =='VSAF']   # Closest OAI contact after the replacement
tDATE = [i for i in dt.columns if i[-4:] =='DATE']   # Date of a replacement
tTLPR = [i for i in dt.columns if i[-4:] =='TLPR']   # Total or Partial replacement indication
tTPPR = [i for i in dt.columns if i[-4:] =='TPPR']   # Type of Partial replacement indication
tPODX = [i for i in dt.columns if i[-4:] =='PODX']   # Primary pre-operative diagnosis
tDEATH = [i for i in dt.columns if i[-4:] =='EDDCF'] # Death columns

DT_outcome = dt[['id'] + tRPCF + tBLRP + tPODX + tVSPR + tDEATH]

In [5]:
DT_outcome.head()

,id,V99ERKRPCF,V99ELKRPCF,V99ERHRPCF,V99ELHRPCF,V99ERKBLRP,V99ELKBLRP,V99ERHBLRP,V99ELHBLRP,V99ERKPODX,V99ELKPODX,V99ERHPODX,V99ELHPODX,V99ERKVSPR,V99ELKVSPR,V99ERHVSPR,V99ELHVSPR,V99EDDVSPR
0,9000099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9000296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9000622,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,9000798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9001104,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN


#### Grouping features names by affected joint ...

In [6]:
tELK = [i for i in DT_outcome.columns if i[0:6] =='V99ELK']  # Left Knee Columns
tERK = [i for i in DT_outcome.columns if i[0:6] =='V99ERK']  # Right Knee Columns
tELH = [i for i in DT_outcome.columns if i[0:6] =='V99ELH']  # Left Hip Columns
tERH = [i for i in DT_outcome.columns if i[0:6] =='V99ELH']  # Right Hip Columns
DT_outcome[['id'] + tERH].head(5)

,id,V99ELHRPCF,V99ELHBLRP,V99ELHPODX,V99ELHVSPR
0,9000099,NaN,NaN,NaN,NaN
1,9000296,NaN,NaN,NaN,NaN
2,9000622,NaN,0.0,NaN,NaN
3,9000798,NaN,NaN,NaN,NaN
4,9001104,3.0,0.0,4.0,3.0


In [7]:
targets = DT_outcome.fillna(0)
targets.describe()

,id,V99ERKRPCF,V99ELKRPCF,V99ERHRPCF,V99ELHRPCF,V99ERKBLRP,V99ELKBLRP,V99ERHBLRP,V99ELHBLRP,V99ERKPODX,V99ELKPODX,V99ERHPODX,V99ELHPODX,V99ERKVSPR,V99ELKVSPR,V99ERHVSPR,V99ELHVSPR,V99EDDVSPR
count,4.796000e+03,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000
mean,9.513826e+06,0.169725,0.167431,0.072977,0.067973,0.007923,0.005213,0.006047,0.004379,0.065680,0.071309,0.031902,0.035029,0.352585,0.335488,0.139491,0.134696,0.414721
std,2.794781e+05,0.690747,0.686311,0.459963,0.442252,0.088669,0.072018,0.077533,0.066033,0.333187,0.408362,0.263180,0.313783,1.600233,1.534531,1.006929,0.995190,1.813524
min,9.000099e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.283430e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.522042e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.747572e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.999878e+06,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,7.000000,7.000000,8.000000,8.000000,10.000000,10.000000,10.000000,10.000000,12.000000


### AllClinicalDataset

Reading the data for the joint dataset...

Here, we are trying to consider only the __last visit__, prior to a replacement. We're trying to catch information that refers to a condition before the surgery.

So:
1. Reading datasets from files. Dropping some columns we won't use
2. Renaming columns, changing.. VxxKPNL12, VxxKPNLEVY to... KPNL12, KPNLEVY, etc..   and refering the visit number in a separated column, named 'visit'. (So we can compare variable between the visits)
3. Joining with the information of last visit prior to a replacement.  (this generates a dataset that have one line per visit... per each ID)
4. Keeping just the last visit (-1) prior to replacement, or a specific visit, for each ID.

In [34]:
dt0 = pd.read_csv('Comma Delimited Data/allclinical01.csv').drop(['VERSION'],axis=1)
dt1 = pd.read_csv('Comma Delimited Data/allclinical02.csv').drop(['VERSION'],axis=1) 
dt2 = pd.read_csv('Comma Delimited Data/allclinical03.csv').drop(['VERSION'],axis=1)
dt3 = pd.read_csv('Comma Delimited Data/allclinical04.csv').drop(['VERSION'],axis=1) 
dt4 = pd.read_csv('Comma Delimited Data/allclinical05.csv').drop(['VERSION'],axis=1) 
dt5 = pd.read_csv('Comma Delimited Data/allclinical06.csv').drop(['VERSION'],axis=1) 
dt6 = pd.read_csv('Comma Delimited Data/allclinical07.csv').drop(['VERSION'],axis=1) 
dt7 = pd.read_csv('Comma Delimited Data/allclinical08.csv').drop(['VERSION'],axis=1) 
dt8 = pd.read_csv('Comma Delimited Data/allclinical09.csv').drop(['VERSION'],axis=1) 
dt9 = pd.read_csv('Comma Delimited Data/allclinical10.csv').drop(['VERSION'],axis=1) 
dt10 = pd.read_csv('Comma Delimited Data/allclinical11.csv').drop(['VERSION'],axis=1)

##### Renaming columns (removing Vxx to enable direct comparison between visits)

the visit number information is kept in the 'visit' column

In [35]:
dts = [dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7,dt8,dt9,dt10]
for j in range(len(dts)):
    d = dts[j]
    col = list(d.columns)
    for i in range(1,len(col)):
        txtcol = col[i]
        col[i] = txtcol[3:]
    d.columns=col
    d['visit'] = j  
ALLDATA = pd.concat(dts,sort=False)

In [36]:
# getting information about the last contact prior to replacement...
visit_to_keep = -1
if visit_to_keep != -1:
    DT_outcome['prior_visit'] = visit_to_keep
else:
    DT_outcome['prior_visit'] = DT_outcome[tVSPR].max(axis=1).fillna(10) # Last visit prior to replacement...


ALLDATA_VISITS = ALLDATA.merge(DT_outcome[['prior_visit','id']],left_on='ID', right_on='id', how='outer')
ALLDATA_VISITS.head()

,ID,BLDRAW2,ILLPWK2,MULTST2,URINOB1,PLAQHR1,BLUPMN2,HOURSP2,VCOLL2,ILLPWK1,...,ACT37A,ACT37B,ACT37C,ACT37D,ACTNAA,ACTNAB,ACTNAC,ACTNAD,prior_visit,id
0,9000099,NaN,NaN,NaN,1.0,33600.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9000099
1,9000099,NaN,NaN,NaN,1.0,27300.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9000099
2,9000099,NaN,NaN,NaN,1.0,53700.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9000099
3,9000099,NaN,NaN,NaN,1.0,29700.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9000099
4,9000099,NaN,NaN,NaN,1.0,27600.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9000099


In [37]:
all_before = True  # 'True' keeps all visits before visit_to_keep, 'False' keeps only visit_to_keep
if all_before:
    ALLDATA_VISITS = ALLDATA_VISITS[ALLDATA_VISITS['prior_visit'] >= ALLDATA_VISITS['visit']]
else:
    # keeping information of only the last visit
    ALLDATA_VISITS = ALLDATA_VISITS[ALLDATA_VISITS['prior_visit'] == ALLDATA_VISITS['visit']]
    
ALLDATA_VISITS['lifetime_visits'] = np.abs(ALLDATA_VISITS['prior_visit'] - ALLDATA_VISITS['visit'])

In [38]:
print(ALLDATA_VISITS[['visit', 'prior_visit','lifetime_visits','id']].head())
ALLDATA_VISITS = ALLDATA_VISITS.drop(['id','visit','prior_visit'],axis=1)

   visit  prior_visit  lifetime_visits       id
0      0         10.0             10.0  9000099
1      2         10.0              8.0  9000099
2      3         10.0              7.0  9000099
3      4         10.0              6.0  9000099
4      5         10.0              5.0  9000099


In [39]:
#######
# Considering to drop the columns below

tSREPHR = [i for i in ALLDATA_VISITS.columns if i[0:4] =='SREP']   # Ever have replacement surgery where all or part of joint was replaced, self-report
tPSDATE = [i for i in ALLDATA_VISITS.columns if i =='PSDATE']   # Date MRI Safety Screener completed
tSSDATE = [i for i in ALLDATA_VISITS.columns if i =='SSDATE']   # 
tObjects = list(ALLDATA_VISITS.dtypes[ALLDATA_VISITS.dtypes == 'object'].keys())

to_drop = tSREPHR + tPSDATE + tSSDATE + tObjects


ALLDATA_VISITS = ALLDATA_VISITS.drop(to_drop,axis=1)

##### Aggregating all visits by mean, grouped by ID...

In [40]:
#ALLDATA_VISITS = ALLDATA_VISITS.groupby('ID', as_index=False).mean()

# Training and Testing Features

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

In [42]:
target_name = tRPCF  # choosing replacement adjucation features

# merging targets with jointsx dataset
X = ALLDATA_VISITS.merge(DT_outcome[target_name + ['id']], left_on='ID', right_on='id', how='outer')

ids = X['ID']
TARGETS = X[target_name]

X = X.drop(['ID','id'] + target_name,axis=1)
train = X.fillna(-1)
train = pd.get_dummies(train)

train_target = TARGETS.sum(axis=1)   ## summing all the replacements
train_target = train_target.fillna(0)     # if no information, then 0
train_target[train_target >= 1] = 1       # if greater than 0... at least one replacement occurred
print('Targets: ', target_name)
print('Dataset: ', ALLDATA_VISITS.shape)
print('Train dataset:', train.shape)

Targets:  ['V99ERKRPCF', 'V99ELKRPCF', 'V99ERHRPCF', 'V99ELHRPCF']
Dataset:  (40903, 1397)
Train dataset: (40903, 1396)


#### Measuring feature importances

In [43]:
# Create the model with several hyperparameters
model = lgb.LGBMClassifier(objective='binary', boosting_type = 'gbdt', n_estimators = 10000)#, class_weight={0:0.3,1:0.7})

# Initialize an empty array to hold feature importances
feature_importances = np.zeros(train.shape[1])

iterations = 5
# Fit the model twice to avoid overfitting
for i in range(iterations):
    
    # Split into training and validation set
    train_features, valid_features, train_y, valid_y = train_test_split(train, train_target, test_size = 0.25, random_state = i)

    print('======================================================================== ')        
    # Train using early stopping
    model.fit(train_features, train_y, early_stopping_rounds=100, 
              eval_set = [(valid_features, valid_y)], eval_metric = 'auc', verbose = 200)

    print('\n===========================================================\nModel ')        
    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration_)
    vmetrics = confusion_matrix(valid_y,valid_pred)
    print ('TN',vmetrics[0,0],', FN',vmetrics[1,0],', FP',vmetrics[0,1],', TP',vmetrics[1,1])
    fpr, tpr, thresholds = metrics.roc_curve(valid_y, valid_pred)
    print('AUC = ', metrics.auc(fpr, tpr))    
    print ('Acc:', (vmetrics[0,0] + vmetrics[1,1]) / len(valid_y))
    
    print('==============')
    print('\n0 Frequency: ', (1 - valid_y.sum()/len(valid_y)))
   
    print('\n==================================\nBASELINE') 
    valid_pred = np.zeros(len(valid_y))
    baseline = confusion_matrix(valid_y,valid_pred)
    fpr, tpr, thresholds = metrics.roc_curve(valid_y, valid_pred)
    print ('TN',baseline[0,0],', FN',baseline[1,0],', FP',baseline[0,1],', TP',baseline[1,1])
    print('AUC = ', metrics.auc(fpr, tpr))
    print ('Acc:', (baseline[0,0] + baseline[1,1]) / len(valid_y))
    
    # Record the feature importances
    feature_importances += model.feature_importances_

feature_importances /= iterations

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.951414
[400]	valid_0's auc: 0.952643
Early stopping, best iteration is:
[319]	valid_0's auc: 0.953281

Model 
TN 9240 , FN 292 , FP 85 , TP 609
AUC =  0.833400183889
Acc: 0.963133189908

0 Frequency:  0.911891257579

BASELINE
TN 9325 , FN 901 , FP 0 , TP 0
AUC =  0.5
Acc: 0.911891257579
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.953222
Early stopping, best iteration is:
[234]	valid_0's auc: 0.953946

Model 
TN 9169 , FN 360 , FP 82 , TP 615
AUC =  0.810952662082
Acc: 0.956776843341

0 Frequency:  0.904654801486

BASELINE
TN 9251 , FN 975 , FP 0 , TP 0
AUC =  0.5
Acc: 0.904654801486
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.949856
Early stopping, best iteration is:
[158]	valid_0's auc: 0.950249

Model 
TN 9250 , FN 336 , FP 72 , TP 568
AUC =  0.810297459811
Acc: 0.960101701545

0 Frequency:  0.911597887737

BASELINE


In [44]:
feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}
                                  ).sort_values('importance', ascending = False)

# Find the features with zero importance
least_important = list(feature_importances[feature_importances['importance'] <= 0]['feature'])
print('There are %d no important features' % len(least_important))

There are 483 no important features


In [45]:
train = train.drop(columns = least_important)
#test = test.drop(columns=least_important)

print('Resulting Training shape: ', train.shape)
#print('Testing shape: ', test.shape)

Resulting Training shape:  (40903, 913)


## Testing

In [46]:
Testing = False

iterations = 10
# Fit the model twice to avoid overfitting
for i in range(iterations):

    if not Testing:
        model = lgb.LGBMClassifier(objective='binary', boosting_type = 'dart', n_estimators = 10000)#, class_weight={0:0.3,1:0.7})    
    
    print('\n=========================================================')
    # splitting test and train dataset...
    train_features, test_features, train_target_, test_target_ = train_test_split(train, train_target, test_size = 0.5, random_state = i)
    print('train: ', train_features.shape, ' test: ' ,test_features.shape)
    
    # splitting validation and train dataset...
    train_X, valid_X, train_y, valid_y = train_test_split(train_features, train_target_, test_size = 0.25, random_state = i)
    
    if not Testing:
        model.fit(train_X, train_y, early_stopping_rounds=100, eval_set = [(valid_X, valid_y)], eval_metric = 'auc', verbose = 200)


    print('\n==== Testing best iteration ====')
    test_pred = model.predict(test_features, num_iteration=model.best_iteration_)

        
    vmetrics = confusion_matrix(test_target_,test_pred)
    print ('TN',vmetrics[0,0],', FN',vmetrics[1,0],', FP',vmetrics[0,1],', TP',vmetrics[1,1])
    fpr, tpr, thresholds = metrics.roc_curve(test_target_, test_pred)
    print('AUC = ', metrics.auc(fpr, tpr))    
    print ('Acc:', (vmetrics[0,0] + vmetrics[1,1]) / len(test_target_))


train:  (20451, 913)  test:  (20452, 913)
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.944743
[400]	valid_0's auc: 0.947752
Early stopping, best iteration is:
[342]	valid_0's auc: 0.949942

==== Testing best iteration ====
TN 18511 , FN 716 , FP 155 , TP 1070
AUC =  0.795400137671
Acc: 0.957412477997

train:  (20451, 913)  test:  (20452, 913)
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.95578
Early stopping, best iteration is:
[276]	valid_0's auc: 0.957646

==== Testing best iteration ====
TN 18492 , FN 721 , FP 170 , TP 1069
AUC =  0.794048641849
Acc: 0.956434578525

train:  (20451, 913)  test:  (20452, 913)
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.950976
[400]	valid_0's auc: 0.953492
Early stopping, best iteration is:
[348]	valid_0's auc: 0.954849

==== Testing best iteration ====
TN 18482 , FN 733 , FP 165 , TP 1072
AUC =  0.79252860441
Acc: 0.95609231371

